In [1]:
!pip install geocoder
!pip install folium

   ---------------------------------------- 0.0/98.6 kB ? eta -:--:--
   --------------------------------- ------ 81.9/98.6 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 98.6/98.6 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.0 kB ? eta -:--:--
   ------------------------ --------------- 61.4/100.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 100.0/100.0 kB 1.4 MB/s eta 0:00:00


In [1]:
import pandas as pd
import requests
import numpy as np
import geocoder
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas import json_normalize

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")


All Required Libraries Imported!


In [5]:
pip install geopy


   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   -------------------------- ------------- 81.9/125.4 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')

In [6]:
import pandas as pd

# Define column names for the DataFrame
column_names = ['Postalcode', 'Borough', 'Neighborhood']

# Initialize an empty list to store the data
data = []

# Find the table body
content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody

# Iterate through each table row
for tr in table.find_all('tr'):
    # Reset variables for each row
    postcode = 0
    borough = 0
    neighborhood = 0
    
    # Iterate through each table data cell
    for td in tr.find_all('td'):
        if not postcode:
            postcode = td.text
        elif not borough:
            borough = td.text
        elif not neighborhood:
            neighborhood = td.text.strip('\n').replace(']', '')

    # Append the data to the list
    data.append({'Postalcode': postcode, 'Borough': borough, 'Neighborhood': neighborhood})

# Create a DataFrame from the list of data
toronto = pd.DataFrame(data, columns=column_names)

toronto.head()  # Display the first few rows of the DataFrame


,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned


In [7]:

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [8]:
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()


,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned


In [9]:
df.describe()

,Postalcode,Borough,Neighborhood
count,20,20,20
unique,20,20,20
top,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods)
freq,1,1,1


In [10]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [11]:
df.head()

,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned


In [12]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df_2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [13]:
print(df_2.shape)
df_2.head()

(20, 3)


,Postalcode,Borough,Neighborhood
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods)
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park)
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned


In [14]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70890314600007, -79.36614352699996]

In [15]:
# Retrieving Postal Code Co-ordinates
postal_codes = df_2['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [16]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_2['Latitude'] = df_coords['Latitude']
df_2['Longitude'] = df_coords['Longitude']

In [17]:
df_2[df_2.Postalcode == 'M5G']

,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [18]:
df_2.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods),43.653524,-79.383907
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North,43.809380,-79.223440
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park),43.786580,-79.188292
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned,43.767370,-79.179500
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned,43.766570,-79.227730
5,\nM1HScarborough(Cedarbrae)\n\n,\nM2HNorth York(Hillcrest Village)\n\n,M3HNorth York(Bathurst Manor / Wilson Heights ...,44.244715,-79.293276
6,\nM1JScarborough(Scarborough Village)\n\n,\nM2JNorth York(Fairview / Henry Farm / Oriole...,M3JNorth York(Northwood Park / York University),43.740615,-79.208887
7,\nM1KScarborough(Kennedy Park / Ionview / East...,\nM2KNorth York(Bayview Village)\n\n,M3KNorth York(Downsview)East (CFB Toronto),43.711087,-79.269800
8,\nM1LScarborough(Golden Mile / Clairlea / Oakr...,\nM2LNorth York(York Mills / Silver Hills)\n\n,M3LNorth York(Downsview)West,43.696632,-79.282859
9,\nM1MScarborough(Cliffside / Cliffcrest / Scar...,\nM2MNorth York(Willowdale / Newtonbrook)\n\n,M3MNorth York(Downsview)Central,43.653524,-79.383907


In [24]:
from geopy.geocoders import Nominatim

address = 'Scarborough, Toronto'

# Specify a custom user_agent
geolocator = Nominatim(user_agent="scarborough-location-app")

location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geographical Coordinates of Scarborough, Toronto are {}, {}.'.format(latitude_x, longitude_y))


The Geographical Coordinates of Scarborough, Toronto are 43.7729744, -79.2576479.


In [25]:
map_Scarborough = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(df_2['Latitude'], df_2['Longitude'], df_2['Neighborhood']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Scarborough)  
    
map_Scarborough

In [31]:
from geopy.geocoders import Nominatim

address = 'Scarborough, Toronto'

# Specify a custom user_agent
geolocator = Nominatim(user_agent="scarborough-location-app")

location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geographical Coordinates of Scarborough, Toronto are {}, {}.'.format(latitude_x, longitude_y))


The Geographical Coordinates of Scarborough, Toronto are 43.7729744, -79.2576479.


In [28]:
# @hiddel_cell
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # my Foursquare ID
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY
CLIENT_SECRET: 1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B


In [32]:
from geopy.geocoders import Nominatim

# Define the address
address = 'Scarborough, Toronto'

# Specify a custom user_agent
geolocator = Nominatim(user_agent="scarborough-location-app")

# Geocode the address to obtain latitude and longitude
location = geolocator.geocode(address)

# Extract latitude and longitude from the location
latitude_n1 = location.latitude
longitude_n1 = location.longitude

# Define other parameters for Foursquare API
radius = 700
LIMIT = 100

# Construct the Foursquare API URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    latitude_n1,
    longitude_n1,
    radius,
    LIMIT)

# Now you can use the URL for further processing with Foursquare API
results = requests.get(url).json()


In [33]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.crossStreet',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.labeledLatLngs', 'venue.location.distance',
       'venue.location.postalCode', 'venue.location.cc', 'venue.location.city',
       'venue.location.state', 'venue.location.country',
       'venue.location.formattedAddress', 'venue.categories',
       'venue.photos.count', 'venue.photos.groups', 'venue.createdAt',
       'photo.id', 'photo.createdAt', 'photo.prefix', 'photo.suffix',
       'photo.width', 'photo.height', 'photo.visibility',
       'venue.location.neighborhood', 'venue.events.count',
       'venue.events.summary', 'venue.venuePage.id'],
      dtype='object')

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,American Eagle Store,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.776012,-79.258334
1,Shoppers Drug Mart,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",43.773305,-79.251662
2,Tommy Hilfiger,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",43.776015,-79.257369
3,Coliseum Scarborough Cinemas,"[{'id': '4bf58dd8d48988d17f941735', 'name': 'M...",43.775995,-79.255649
4,St. Louis Bar & Grill,"[{'id': '4bf58dd8d48988d11d941735', 'name': 'S...",43.774157,-79.253808


In [36]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(5)

,name,categories,lat,lng
0,American Eagle Store,Clothing Store,43.776012,-79.258334
1,Shoppers Drug Mart,Pharmacy,43.773305,-79.251662
2,Tommy Hilfiger,Clothing Store,43.776015,-79.257369
3,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
4,St. Louis Bar & Grill,Sports Bar,43.774157,-79.253808


In [37]:
# Top 10 Categories
a=pd.Series(nearby_venues.categories)
a.value_counts()[:10]

categories
Clothing Store         8
Coffee Shop            4
Restaurant             3
Fuel Station           3
Electronics Store      2
Sandwich Spot          2
Pharmacy               2
Department Store       2
Fish and Chips Shop    1
Discount Store         1
Name: count, dtype: int64

In [38]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [39]:
# Nearby Venues
Scarborough_venues = getNearbyVenues(names=df_2['Neighborhood'],
                                   latitudes=df_2['Latitude'],
                                   longitudes=df_2['Longitude']
                                  )

M3ANorth York(Parkwoods)
M3BNorth York(Don Mills)North
M3CNorth York(Don Mills)South(Flemingdon Park)
M3ENot assigned
M3GNot assigned
M3HNorth York(Bathurst Manor / Wilson Heights / Downsview North)
M3JNorth York(Northwood Park / York University)
M3KNorth York(Downsview)East  (CFB Toronto)
M3LNorth York(Downsview)West
M3MNorth York(Downsview)Central
M3NNorth York(Downsview)Northwest
M3PNot assigned
M3RNot assigned
M3SNot assigned
M3TNot assigned
M3VNot assigned
M3WNot assigned
M3XNot assigned
M3YNot assigned
M3ZNot assigned


In [40]:
print('There are {} Uniques Categories.'.format(len(Scarborough_venues['Venue Category'].unique())))
Scarborough_venues.groupby('Neighborhood').count().head()

There are 97 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
M3ANorth York(Parkwoods),100,100,100,100,100,100
M3BNorth York(Don Mills)North,15,15,15,15,15,15
M3CNorth York(Don Mills)South(Flemingdon Park),9,9,9,9,9,9
M3ENot assigned,20,20,20,20,20,20
M3GNot assigned,8,8,8,8,8,8


In [41]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_onehot.head(5)

,Vietnamese Restaurant,American Restaurant,Arts and Crafts Store,Arts and Entertainment,Asian Restaurant,Bakery,Bank,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Bridge,Bubble Tea Shop,Burger Joint,Burrito Restaurant,Bus Stop,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Church,City Hall,Clothing Store,Coffee Shop,Comic Book Store,Concert Hall,Convenience Store,Cosmetics Store,Courthouse,Deli,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food and Beverage Retail,Fried Chicken Joint,Fuel Station,Furniture and Home Store,Gastropub,Greek Restaurant,Grocery Store,Hair Salon,Halal Restaurant,Hiking Trail,Hobby Store,Hong Kong Restaurant,Hotel,Ice Cream Parlor,Indian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Liquor Store,Marijuana Dispensary,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Store,Modern European Restaurant,Monument,Movie Theater,Music Venue,Neighborhood,New American Restaurant,Park,Pet Supplies Store,Pharmacy,Pizzeria,Plaza,Pub,Ramen Restaurant,Restaurant,Sandwich Spot,Seafood Restaurant,Shopping Mall,Spa,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Taco Restaurant,Tanning Salon,Thai Restaurant,Theater,University,Vegan and Vegetarian Restaurant,Video Games Store
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,M3ANorth York(Parkwoods),False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,M3ANorth York(Parkwoods),False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,M3ANorth York(Parkwoods),False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,M3ANorth York(Parkwoods),False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fa

In [42]:
num_top_venues = 5
for hood in Scarborough_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- M3ANorth York(Parkwoods) ----
                 venue  freq
0          Coffee Shop  0.08
1                Hotel  0.06
2  Japanese Restaurant  0.04
3       Clothing Store  0.04
4                 Café  0.04


---- M3BNorth York(Don Mills)North ----
                  venue  freq
0  Fast Food Restaurant  0.20
1                  Park  0.13
2              Pharmacy  0.13
3              Pizzeria  0.13
4     Convenience Store  0.07


---- M3CNorth York(Don Mills)South(Flemingdon Park) ----
                 venue  freq
0         Burger Joint  0.22
1          Coffee Shop  0.22
2         Fuel Station  0.22
3                  Pub  0.11
4  Fried Chicken Joint  0.11


---- M3ENot assigned ----
           venue  freq
0       Pizzeria  0.15
1           Park  0.10
2    Coffee Shop  0.10
3       Pharmacy  0.10
4  Shopping Mall  0.05


---- M3GNot assigned ----
                   venue  freq
0  Vietnamese Restaurant  0.12
1        Taco Restaurant  0.12
2      Indian Restaurant  0.12
3   Fast Food Rest

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3ANorth York(Parkwoods),Coffee Shop,Hotel,Japanese Restaurant,Clothing Store,Café,Monument,Steakhouse,Breakfast Spot,Cosmetics Store,American Restaurant
1,M3BNorth York(Don Mills)North,Fast Food Restaurant,Park,Pharmacy,Pizzeria,Convenience Store,Sandwich Spot,Hair Salon,Grocery Store,Supermarket,Bubble Tea Shop
2,M3CNorth York(Don Mills)South(Flemingdon Park),Burger Joint,Coffee Shop,Fuel Station,Pub,Fried Chicken Joint,Mediterranean Restaurant,Latin American Restaurant,Monument,Modern European Restaurant,Miscellaneous Store
3,M3ENot assigned,Pizzeria,Park,Coffee Shop,Pharmacy,Shopping Mall,Fried Chicken Joint,Food and Beverage Retail,Liquor Store,Filipino Restaurant,Fast Food Restaurant
4,M3GNot assigned,Vietnamese Restaurant,Taco Restaurant,Indian Restaurant,Fast Food Restaurant,Discount Store,Park,Supermarket,Coffee Shop,Sushi Restaurant,Modern European Restaurant


In [51]:
# Using K-Means to cluster neighborhood into 3 clusters
Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters=2, random_state=0).fit(Scarborough_grouped_clustering)
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1])

In [56]:
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



Scarborough_merged =df_2.iloc[:16,:]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head()# check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M3ANorth York(Parkwoods),43.653524,-79.383907,0.0,Coffee Shop,Hotel,Japanese Restaurant,Clothing Store,Café,Monument,Steakhouse,Breakfast Spot,Cosmetics Store,American Restaurant
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M3BNorth York(Don Mills)North,43.809380,-79.223440,0.0,Fast Food Restaurant,Park,Pharmacy,Pizzeria,Convenience Store,Sandwich Spot,Hair Salon,Grocery Store,Supermarket,Bubble Tea Shop
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M3CNorth York(Don Mills)South(Flemingdon Park),43.786580,-79.188292,0.0,Burger Joint,Coffee Shop,Fuel Station,Pub,Fried Chicken Joint,Mediterranean Restaurant,Latin American Restaurant,Monument,Modern European Restaurant,Miscellaneous Store
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M3ENot assigned,43.767370,-79.179500,0.0,Pizzeria,Park,Coffee Shop,Pharmacy,Shopping Mall,Fried Chicken Joint,Food and Beverage Retail,Liquor Store,Filipino Restaurant,Fast Food Restaurant
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M3GNot assigned,43.766570,-79.227730,0.0,Vietnamese Restaurant,Taco Restaurant,Indian Restaurant,Fast Food Restaurant,Discount Store,Park,Supermarket,Coffee Shop,Sushi Restaurant,Modern European Restaurant


In [57]:
kclusters = 10

In [63]:
# Drop rows with missing values in 'Cluster Labels' column
Scarborough_merged.dropna(subset=['Cluster Labels'], inplace=True)

# Create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# Add markers to the map
markers_colors = []
for lat, lon, nei, cluster in zip(Scarborough_merged['Latitude'], 
                                   Scarborough_merged['Longitude'], 
                                   Scarborough_merged['Neighborhood'], 
                                   Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']


In [64]:
df1=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]
df2=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]
df3=Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

In [65]:
Scarborough_Avg_HousingPrice=pd.DataFrame({"Neighborhood":df_2["Neighborhood"],
                                      "Average_Housing_Price":[335000.0,286600.0,175000.0,225900.0,219400.0,
                                                               573900.0,225000.0,370500.0,370500.0,433500.0,279200.0,
                                                               279200.0,225000.0,370500.,255400.0,433500.0,433500.0,
                                                               435000.0,289500.0,265000.0,285900.0,239400.0,
                                                               589900.0,295000.0,380500.0,378500.0,438500.0,229200.0,
                                                               229200.0,365000.0,388500.,285400.0,493500.0,477500.0,378000.0,316600.0,195000.0,225900.0,219400.0,
                                                               573900.0,367000.0,370500.0,370500.0,363500.0,279200.0,
                                                               279200.0,271000.0,370500.,255400.0,383500.0,433500.0,335000.0,286600.0,185000.0,225900.0,219400.0,
                                                               573900.0,329000.0,370500.0,370500.0,533500.0,279200.0,
                                                               279200.0,375000.0,370500.,255400.0,493500.0,433500.0,335000.0,286600.0,165000.0,225900.0,219400.0,
                                                               573900.0,425000.0,370500.0,370500.0,433500.0,279200.0,
                                                               279200.0,195000.0,370500.,255400.0,403500.0,433500.0,335000.0,286600.0,187000.0,225900.0,219400.0,
                                                               573900.0,325000.0,370500.0,370500.0,333500.0,279200.0,
                                                               279200.0,289000.0,370500.,255400.0,413500.0,433500.0,254800.0
                                                              
                                                              
                                                              ]})

ValueError: array length 103 does not match index length 20

In [66]:
Scarborough_Avg_HousingPrice.set_index('Neighborhood',inplace=True,drop=True)

NameError: name 'Scarborough_Avg_HousingPrice' is not defined

In [67]:
Scarborough_Avg_HousingPrice.plot(kind='bar',figsize=(24,18),alpha=0.75)

NameError: name 'Scarborough_Avg_HousingPrice' is not defined

In [68]:
clusters=pd.DataFrame({"Cluster1":df1["Neighborhood"],
                      "Cluster2":df2["Neighborhood"],
                      "Cluster4":df3["Neighborhood"]})
clusters = clusters.replace(np.nan, '', regex=True)

In [69]:
new_Scarborough=Scarborough_merged.set_index("Neighborhood",drop=True)
#Source:https://www.greatschools.org
Scarborough_school_ratings=pd.DataFrame({"Neighborhood":df["Neighborhood"],
                                      "Top School Rating":[7,9,5,8,10,10,7,10,1,2,1,2,7,2,3,2,6,
                                                           5,4,8,9,9,6,6,4,5,4,6,8,10,8,9,6,2,
                                                           10,2,5,8,9,6,6,10,8,9,1,2,3,4,5,6,9,
                                                           8,5,9,6,9,6,4,8,10,2,5,6,3,9,8,7,
                                                           7,8,5,8,9,1,5,4,7,2,3,6,6,9,4,8,7,
                                                           4,8,9,2,6,4,7,5,10,4,6,8,9,7,5,6,5,8,7
                                                           ]})

ValueError: array length 103 does not match index length 20